# 3. Creating Table 1 for the Therlid Dataset

Table 1 creation process stratified by various variables within the Therlid dataset

## Defines and Imports

In [ ]:
# replace with own projectid to pull from bigquery
%env GOOGLE_CLOUD_PROJECT = aiwonglab

env: GOOGLE_CLOUD_PROJECT=aiwonglab


In [ ]:
# needs to be this version of tableone
!pip install tableone==0.8.0
# !pip install tableone

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
from google.cloud import bigquery
import warnings
warnings.filterwarnings('ignore')

import missingno as msno
import matplotlib.pyplot as plt

import numpy as np
from tableone import TableOne

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


In [ ]:
%config InlineBackend.figure_format = 'retina'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/matplotlib_inline/config.py:68: DeprecationWarning: InlineBackend._figure_format_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_format_changed(self, name, old, new):


In [ ]:
%%bigquery final_df --project spring-carving-271217

SELECT *
FROM `spring-carving-271217.test.merged_temp_pairs`

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
final_df["region"].unique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['Midwest', 'Northeast', None, 'South', 'West'], dtype=object)

## Preprocess

In [ ]:
vitals =      [c for c in final_df.columns if c.startswith('vitals_')]
cbc =         [c for c in final_df.columns if c.startswith('cbc_')]
coag =        [c for c in final_df.columns if c.startswith('coag_')]
bmp =         [c for c in final_df.columns if c.startswith('bmp_')]
hfp =         [c for c in final_df.columns if c.startswith('hfp_')]
others =      [c for c in final_df.columns if c.startswith('others_')]
sofa_past =   [c for c in final_df.columns if c.startswith('sofa_past_')]
sofa_future = [c for c in final_df.columns if c.startswith('sofa_future_')]
deltas =      [c for c in final_df.columns if c.startswith('delta_')]
abgs =        [c for c in final_df.columns if c.startswith('abg_')]
temp_pairs = ["temp_c_reference", "temp_c_temporal", "pair_type", "total_no_pairs"]

pat_categ = ["hidden_SIRS", "hidden_fever", "hidden_hypothermia", "source_db", "region", "sex_female", "race_ethnicity"]

pat_cont = ["admission_age","weight_admission", "height_admission",
            "BMI_admission","comorbidity_score_value", 'sofa_past_overall_24hr', 'sofa_future_overall_24hr', 'sofa_day1', "los_hosp_dead", "los_hosp_surv",
            "los_ICU_dead", "los_ICU_surv", "total_no_pairs"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# binary category
final_df['race_ethnicity_white'] = final_df['race_ethnicity'] \
                              .apply(lambda x: "White" if x == "White" \
                                       else "Racial-Ethnic Group")

# continuous variables
final_df['los_hosp_dead'] = final_df[final_df.in_hospital_mortality == 1].los_hospital
final_df['los_hosp_surv'] = final_df[final_df.in_hospital_mortality == 0].los_hospital
final_df['los_ICU_dead'] = final_df[final_df.in_hospital_mortality == 1].los_ICU
final_df['los_ICU_surv'] = final_df[final_df.in_hospital_mortality == 0].los_ICU

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
x = final_df[final_df["source_db"] == "mimic_iv"]

pd.crosstab(
    x['race_ethnicity'],
    x['sex_female']
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


sex_female,0,1
race_ethnicity,,
American Indian / Alaska Native,5,1
Asian,29,37
Black,75,65
Hispanic OR Latino,32,35
Native Hawaiian / Pacific Islander,11,4
Unknown,177,115
White,582,384


In [ ]:
(pd.crosstab(
    x['race_ethnicity'],
    x['sex_female'],
    normalize=True
) * 100).round(2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


sex_female,0,1
race_ethnicity,,
American Indian / Alaska Native,0.32,0.06
Asian,1.87,2.38
Black,4.83,4.19
Hispanic OR Latino,2.06,2.26
Native Hawaiian / Pacific Islander,0.71,0.26
Unknown,11.40,7.41
White,37.50,24.74


## Straitfy by Source DB

### Table One by Patient Level


In [ ]:
groupby = ["source_db"]
categ = list(set(pat_categ) - set(groupby))
nonnorm = pat_cont
cols = categ + nonnorm

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# double check
data_pat = final_df.groupby(by=["unique_subject_id", "hospital_admission_id"]).first().reset_index()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
table1_pat = TableOne(data_pat, columns=cols,
                      groupby=groupby, categorical=categ, nonnormal=nonnorm,
                      missing=True, overall=False, pval=True, htest_name=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
table1_pat

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Grouped by source_db                                                                                                                  
                                                                                         Missing                 eicu                eicu2             mimic_iv P-Value                                       Test
n                                                                                                                4978                 2764                  937                                                   
race_ethnicity, n (%)                    American Indian / Alaska Native                       0             34 (0.7)             19 (0.7)              3 (0.3)  <0.001  Chi-squared (warning: expected count < 5)
                                         Asian                                                               46 (0.9)             71 (2.6)             34 (3.6)                                                   
                                         Black                                                              310 (6.2)           433 (15.7)             84 (9.0)                                                   
                                         Hispanic OR Latino                                                 151 (3.0)             50 (1.8)             41 (4.4)                                                   
                                         Unknown                                                            214 (4.3)            187 (6.8)           191 (20.4)                                                   
                                         White                                                            4223 (84.8)          2004 (72.5)           578 (61.7)                                                   
                                         Native Hawaiian / Pacific Islander                                                                             6 (0.6)                                                   
hidden_hypothermia, n (%)                False                                                 0          4810 (96.6)          2683 (97.1)           896 (95.6)   0.102                                Chi-squared
                                         True                                                               168 (3.4)             81 (2.9)             41 (4.4)                                                   
sex_female, n (%)                        0                                                     0          2745 (55.1)          1481 (53.6)           525 (56.0)   0.294                                Chi-squared
                                         1                                                                2233 (44.9)          1283 (46.4)           412 (44.0)                                                   
hidden_SIRS, n (%)                       False                                                 0          4647 (93.4)          2599 (94.0)           851 (90.8)   0.003                                Chi-squared
                                         True                                                               331 (6.6)            165 (6.0)             86 (9.2)                                                   
region, n (%)                            Midwest                                             324          2664 (57.2)          1357 (49.1)                       <0.001                                Chi-squared
                                         Northeast                                                           11 (0.2)           567 (20.5)          937 (100.0)                                                   
                                         South                                                             586 (12.6)           631 (22.8)                                                                        
                                         West                      

###  Table One by Pairs Level

In [ ]:
groupby = ["source_db"]
categ = ["hidden_SIRS", "hidden_fever", "hidden_hypothermia"]
nonnorm =  abgs + vitals + cbc + coag + bmp + hfp + others + sofa_past + sofa_future
cols = categ + nonnorm

In [ ]:
table1_pair = TableOne(final_df, columns=cols,
                       groupby=groupby, categorical=categ, nonnormal=nonnorm,
                       missing=True, overall=False)

In [ ]:
with pd.option_context('display.max_columns', None, 'display.max_rows',None):
  display(table1_pair)

Grouped by source_db                                                               
                                                                   Missing                 eicu                eicu2             mimic_iv
n                                                                                          7337                 4392                 1552
hidden_SIRS, n (%)                              False                    0          6853 (93.4)          4143 (94.3)          1414 (91.1)
                                                True                                  484 (6.6)            249 (5.7)            138 (8.9)
hidden_fever, n (%)                             False                    0          7096 (96.7)          4276 (97.4)          1481 (95.4)
                                                True                                  241 (3.3)            116 (2.6)             71 (4.6)
hidden_hypothermia, n (%)                       False                    0          7094 (96.7)          4259 (97.0)          1485 (95.7)
                                                True                                  243 (3.3)            133 (3.0)             67 (4.3)
abg_Carboxyhemoglobin, median [Q1,Q3]                                13058        1.0 [0.7,1.6]        0.6 [0.3,1.1]        1.0 [0.8,1.2]
abg_Methemoglobin, median [Q1,Q3]                                    13043        0.4 [0.3,0.8]        0.3 [0.3,0.6]        1.0 [0.5,1.0]
abg_SaO2, median [Q1,Q3]                                             11805     97.4 [95.7,99.0]     98.0 [95.0,99.1]     96.0 [95.0,97.0]
abg_pH, median [Q1,Q3]                                               11188        7.4 [7.3,7.4]        7.4 [7.3,7.4]        7.4 [7.3,7.4]
abg_paCO2, median [Q1,Q3]                                            11174     39.0 [34.0,44.1]     38.0 [34.0,44.0]     40.0 [34.0,44.0]
abg_paO2, median [Q1,Q3]                                             11167   136.0 [86.0,227.0]   130.0 [82.0,192.0]   109.0 [89.0,144.0]
vitals_heart_rate_min, median [Q1,Q3]                                  757     84.0 [72.0,97.0]     84.0 [72.0,96.0]    87.0 [74.0,105.0]
vitals_resp_rate, median [Q1,Q3]                                       151     18.0 [16.0,20.0]     18.0 [16.0,20.0]     20.0 [16.0,25.0]
vitals_mbp_ni, median [Q1,Q3]                                         2729     85.0 [74.0,96.0]     89.0 [79.0,99.0]     75.0 [66.0,87.0]
vitals_sbp_ni, median [Q1,Q3]                                         1238  125.0 [110.0,142.0]  125.0 [111.0,141.0]   113.5 [99.0,128.0]
vitals_dbp_ni, median [Q1,Q3]                                         1238     68.0 [59.0,78.0]     68.0 [60.0,78.0]     62.0 [53.0,73.0]
vitals_mbp_i, median [Q1,Q3]                                          8769     85.0 [74.0,96.0]     84.0 [74.0,94.0]     74.0 [66.0,84.0]
vitals_sbp_i, median [Q1,Q3]                                          9068  128.0 [111.0,145.8]  128.0 [112.0,143.0]  112.0 [100.0,127.0]
vitals_dbp_i, median [Q1,Q3]                                          9070     61.0 [53.0,70.0]     62.0 [53.0,71.0]     56.0 [49.0,64.0]
vitals_tempc_0_2h_before, median [Q1,Q3]                              5700     36.7 [36.4,37.2]     36.6 [36.3,37.0]     36.8 [36.2,37.4]
vitals_tempc_2h_4h_before, median [Q1,Q3]                             6478     36.8 [36.4,37.2]     36.7 [36.4,37.0]     36.9 [36.4,37.4]
vitals_tempc_4h_8h_before, median [Q1,Q3]                             4562     36.8 [36.5,37.1]     36.7 [36.5,37.0]     36.9 [36.4,37.3]
vitals_tempc_8h_12h_before, median [Q1,Q3]                            5436     36.8 [36.6,37.1]     36.7 [36.5,37.0]     36.9 [36.5,37.4]
vitals_tempc_12h_20h_before, median [Q1,Q3]                           5110     36.8 [36.6,37.2]     36.8 [36.5,37.0]     36.9 [36.6,37.3]
vitals_tempc_20h_28h_before, median [Q1,Q3]                           6057     36.8 [36.5,37.1]     36.8 [36.5,37.0]     36.9 [36.6,37.3]
vitals_tempc_28h_36h_before, median [Q1,Q3]         

## Stratify by Race and Ethnicity

### Table One by Patient Level

In [ ]:
groupby = ["race_ethnicity"]
categ = list(set(pat_categ) - set(groupby))
nonnorm = pat_cont
cols = categ + nonnorm

data_pat = final_df.groupby(by=["unique_subject_id", "hospital_admission_id"]).first().reset_index()

table1_pat = TableOne(data_pat, columns=cols,
                      groupby=groupby, categorical=categ, nonnormal=nonnorm,
                      missing=True, overall=False)

with pd.option_context('display.max_columns', None, 'display.max_rows',None):
  display(table1_pat)

Grouped by race_ethnicity                                                                                                                                                                            
                                                                     Missing American Indian / Alaska Native                Asian                Black   Hispanic OR Latino Native Hawaiian / Pacific Islander              Unknown                White
n                                                                                                         56                  151                  827                  242                                  6                  592                 6805
hidden_hypothermia, n (%)                False                             0                       54 (96.4)           143 (94.7)           789 (95.4)           234 (96.7)                          6 (100.0)           570 (96.3)          6593 (96.9)
                                         True                                                        2 (3.6)              8 (5.3)             38 (4.6)              8 (3.3)                                                22 (3.7)            212 (3.1)
sex_female, n (%)                        0                                 0                       32 (57.1)            82 (54.3)           415 (50.2)           124 (51.2)                           3 (50.0)           341 (57.6)          3754 (55.2)
                                         1                                                         24 (42.9)            69 (45.7)           412 (49.8)           118 (48.8)                           3 (50.0)           251 (42.4)          3051 (44.8)
hidden_SIRS, n (%)                       False                             0                       49 (87.5)           135 (89.4)           751 (90.8)           221 (91.3)                          6 (100.0)           534 (90.2)          6401 (94.1)
                                         True                                                       7 (12.5)            16 (10.6)             76 (9.2)             21 (8.7)                                                58 (9.8)            404 (5.9)
region, n (%)                            Midwest                         324                       21 (37.5)            40 (27.0)           294 (36.1)            41 (17.7)                                               95 (16.4)          3530 (54.2)
                                         Northeast                                                  7 (12.5)            70 (47.3)           157 (19.3)            82 (35.5)                          6 (100.0)           290 (49.9)           903 (13.9)
                                         South                                                     12 (21.4)            21 (14.2)           326 (40.0)             14 (6.1)                                               71 (12.2)           773 (11.9)
                                         West                                                      16 (28.6)            17 (11.5)             38 (4.7)            94 (40.7)                                              125 (21.5)          1312 (20.1)
hidden_fever, n (%)                      False                             0                       51 (91.1)           143 (94.7)           789 (95.4)           229 (94.6)                          6 (100.0)           556 (93.9)          6613 (97.2)
                                         True                                                        5 (8.9)              8 (5.3)             38 (4.6)             13 (5.4)                                                36 (6.1)            192 (2.8)
source_db, n (%)                         eicu                              0                       34 (60.7)            46 (30.5)           310 (37.5)           151 (62.4)                                              214 (36.1)          4223 (62.1)
                                         eicu2                     

### Table One by Pair Level

In [ ]:
groupby = ["race_ethnicity_white"]
categ = ["hidden_SIRS", "hidden_fever", "hidden_hypothermia"]
nonnorm =  abgs + vitals + cbc + coag + bmp + hfp + others + sofa_past + sofa_future
cols = categ + nonnorm

table1_pair = TableOne(final_df, columns=cols,
                       groupby=groupby, categorical=categ, nonnormal=nonnorm,
                       missing=True, overall=False)

with pd.option_context('display.max_columns', None, 'display.max_rows',None):
  display(table1_pair)

Grouped by race_ethnicity_white                                          
                                                                              Missing  Racial-Ethnic Group                White
n                                                                                                     2900                10381
hidden_SIRS, n (%)                              False                               0          2640 (91.0)          9770 (94.1)
                                                True                                             260 (9.0)            611 (5.9)
hidden_fever, n (%)                             False                               0          2758 (95.1)         10095 (97.2)
                                                True                                             142 (4.9)            286 (2.8)
hidden_hypothermia, n (%)                       False                               0          2782 (95.9)         10056 (96.9)
                                                True                                             118 (4.1)            325 (3.1)
abg_Carboxyhemoglobin, median [Q1,Q3]                                           13058        0.8 [0.3,1.0]        1.0 [0.5,1.6]
abg_Methemoglobin, median [Q1,Q3]                                               13043        0.3 [0.3,0.4]        0.5 [0.3,0.9]
abg_SaO2, median [Q1,Q3]                                                        11805     97.0 [95.0,99.0]     97.3 [95.4,99.0]
abg_pH, median [Q1,Q3]                                                          11188        7.4 [7.3,7.4]        7.4 [7.3,7.4]
abg_paCO2, median [Q1,Q3]                                                       11174     38.0 [34.0,44.0]     39.0 [34.0,44.0]
abg_paO2, median [Q1,Q3]                                                        11167   122.0 [81.0,189.0]   132.0 [86.0,204.2]
vitals_heart_rate_min, median [Q1,Q3]                                             757    86.0 [73.0,101.0]     84.0 [72.0,97.0]
vitals_resp_rate, median [Q1,Q3]                                                  151     18.0 [16.0,22.0]     18.0 [16.0,21.0]
vitals_mbp_ni, median [Q1,Q3]                                                    2729     87.0 [76.0,99.0]     86.0 [75.0,96.0]
vitals_sbp_ni, median [Q1,Q3]                                                    1238  125.0 [109.0,143.0]  124.0 [110.0,140.0]
vitals_dbp_ni, median [Q1,Q3]                                                    1238     69.0 [60.0,79.0]     67.0 [58.0,77.0]
vitals_mbp_i, median [Q1,Q3]                                                     8769     82.0 [71.0,93.0]     82.0 [72.0,93.0]
vitals_sbp_i, median [Q1,Q3]                                                     9068  121.0 [106.0,140.0]  125.0 [109.0,142.0]
vitals_dbp_i, median [Q1,Q3]                                                     9070     61.0 [51.0,70.0]     60.0 [52.0,69.0]
vitals_tempc_0_2h_before, median [Q1,Q3]                                         5700     36.7 [36.3,37.2]     36.7 [36.3,37.1]
vitals_tempc_2h_4h_before, median [Q1,Q3]                                        6478     36.8 [36.4,37.2]     36.8 [36.4,37.2]
vitals_tempc_4h_8h_before, median [Q1,Q3]                                        4562     36.8 [36.5,37.2]     36.8 [36.5,37.1]
vitals_tempc_8h_12h_before, median [Q1,Q3]                                       5436     36.8 [36.6,37.2]     36.8 [36.5,37.1]
vitals_tempc_12h_20h_before, median [Q1,Q3]                                      5110     36.8 [36.6,37.1]     36.8 [36.6,37.1]
vitals_tempc_20h_28h_before, median [Q1,Q3]                                      6057     36.8 [36.5,37.1]     36.8 [36.5,37.1]
vitals_tempc_28h_36h_before, median [Q1,Q3]                                      6963     36.8 [36.6,37.1]     36.8 [36.5,37.1]
vitals_tempc_36h_48h_before, median [Q1,Q3]                                      7142     36.8 [36.6,37.2]     36.8 [36.6,37.1]
vitals_spo2, median [Q1,Q3]                                                      1270 

## Stratify by Hidden SIRS

### Table One by Patient Level

In [ ]:
groupby = ["hidden_SIRS"]
categ = list(set(pat_categ) - set(groupby))
nonnorm = pat_cont
cols = categ + nonnorm

data_pat = final_df.groupby(by=["unique_subject_id", "hospital_admission_id"]).first().reset_index()

table1_pat = TableOne(data_pat, columns=cols,
                      groupby=groupby, categorical=categ, nonnormal=nonnorm,
                      missing=True, overall=False)

with pd.option_context('display.max_columns', None, 'display.max_rows',None):
  display(table1_pat)

Grouped by hidden_SIRS                                          
                                                                                           Missing                False                 True
n                                                                                                                  8097                  582
hidden_hypothermia, n (%)                False                                                   0         8097 (100.0)           292 (50.2)
                                         True                                                                                     290 (49.8)
sex_female, n (%)                        0                                                       0          4416 (54.5)           335 (57.6)
                                         1                                                                  3681 (45.5)           247 (42.4)
race_ethnicity, n (%)                    American Indian / Alaska Native                         0             49 (0.6)              7 (1.2)
                                         Asian                                                                135 (1.7)             16 (2.7)
                                         Black                                                                751 (9.3)            76 (13.1)
                                         Hispanic OR Latino                                                   221 (2.7)             21 (3.6)
                                         Native Hawaiian / Pacific Islander                                     6 (0.1)                     
                                         Unknown                                                              534 (6.6)            58 (10.0)
                                         White                                                              6401 (79.1)           404 (69.4)
region, n (%)                            Midwest                                               324          3805 (48.8)           216 (38.5)
                                         Northeast                                                          1362 (17.5)           153 (27.3)
                                         South                                                              1121 (14.4)            96 (17.1)
                                         West                                                               1506 (19.3)            96 (17.1)
hidden_fever, n (%)                      False                                                   0         8097 (100.0)           290 (49.8)
                                         True                                                                                     292 (50.2)
source_db, n (%)                         eicu                                                    0          4647 (57.4)           331 (56.9)
                                         eicu2                                                              2599 (32.1)           165 (28.4)
                                         mimic_iv                                                            851 (10.5)            86 (14.8)
admission_age, median [Q1,Q3]                                                                    0     64.0 [52.0,74.0]     65.0 [52.0,75.0]
weight_admission, median [Q1,Q3]                                                               296     81.0 [67.5,97.5]     80.0 [66.0,96.1]
height_admission, median [Q1,Q3]                                                               376  170.2 [162.6,177.8]  170.1 [162.6,177.8]
BMI_admission, median [Q1,Q3]                                                                  555     27.8 [23.9,32.9]     27.5 [23.2,32.2]
comorbidity_score_value, median [Q1,Q3]                                                          0        3.0 [2.0,5.0]        4.0 [2.0,5.0]
sofa_past_overall_24hr, median [Q1,Q3]                                                        5289        4.0 [2.0,7.0]        4

### Table One by Pair Level

In [ ]:
groupby = ["hidden_SIRS"]
categ = ["hidden_fever", "hidden_hypothermia"]
nonnorm =  abgs + vitals + cbc + coag + bmp + hfp + others + sofa_past + sofa_future
cols = categ + nonnorm

table1_pair = TableOne(final_df, columns=cols,
                       groupby=groupby, categorical=categ, nonnormal=nonnorm,
                       missing=True, overall=False)

with pd.option_context('display.max_columns', None, 'display.max_rows',None):
  display(table1_pair)

Grouped by hidden_SIRS                                          
                                                                     Missing                False                 True
n                                                                                           12410                  871
hidden_fever, n (%)                             False                      0        12410 (100.0)           443 (50.9)
                                                True                                                        428 (49.1)
hidden_hypothermia, n (%)                       False                      0        12410 (100.0)           428 (49.1)
                                                True                                                        443 (50.9)
abg_Carboxyhemoglobin, median [Q1,Q3]                                  13058        0.9 [0.4,1.4]        0.7 [0.3,1.1]
abg_Methemoglobin, median [Q1,Q3]                                      13043        0.4 [0.3,0.7]        0.4 [0.3,0.6]
abg_SaO2, median [Q1,Q3]                                               11805     97.3 [95.1,99.0]     97.0 [95.5,98.5]
abg_pH, median [Q1,Q3]                                                 11188        7.4 [7.3,7.4]        7.4 [7.3,7.4]
abg_paCO2, median [Q1,Q3]                                              11174     38.2 [34.0,44.0]     41.0 [33.0,47.0]
abg_paO2, median [Q1,Q3]                                               11167   132.0 [86.0,204.5]    97.0 [75.0,155.0]
vitals_heart_rate_min, median [Q1,Q3]                                    757     84.0 [72.0,98.0]    86.0 [74.0,104.0]
vitals_resp_rate, median [Q1,Q3]                                         151     18.0 [16.0,21.0]     19.0 [16.0,23.0]
vitals_mbp_ni, median [Q1,Q3]                                           2729     86.0 [76.0,97.0]     83.0 [72.0,96.0]
vitals_sbp_ni, median [Q1,Q3]                                           1238  124.0 [110.0,141.0]  121.0 [104.0,140.0]
vitals_dbp_ni, median [Q1,Q3]                                           1238     68.0 [59.0,78.0]     65.0 [56.0,76.0]
vitals_mbp_i, median [Q1,Q3]                                            8769     82.0 [72.0,93.0]     80.0 [69.0,92.0]
vitals_sbp_i, median [Q1,Q3]                                            9068  125.0 [109.0,142.0]  117.0 [102.8,142.2]
vitals_dbp_i, median [Q1,Q3]                                            9070     60.0 [52.0,69.0]     58.0 [50.0,66.0]
vitals_tempc_0_2h_before, median [Q1,Q3]                                5700     36.7 [36.4,37.1]     36.7 [36.1,37.9]
vitals_tempc_2h_4h_before, median [Q1,Q3]                               6478     36.8 [36.4,37.1]     36.8 [36.3,37.6]
vitals_tempc_4h_8h_before, median [Q1,Q3]                               4562     36.8 [36.5,37.1]     36.9 [36.4,37.5]
vitals_tempc_8h_12h_before, median [Q1,Q3]                              5436     36.8 [36.6,37.1]     36.8 [36.4,37.4]
vitals_tempc_12h_20h_before, median [Q1,Q3]                             5110     36.8 [36.6,37.1]     36.8 [36.3,37.4]
vitals_tempc_20h_28h_before, median [Q1,Q3]                             6057     36.8 [36.5,37.1]     36.8 [36.4,37.4]
vitals_tempc_28h_36h_before, median [Q1,Q3]                             6963     36.8 [36.6,37.1]     36.8 [36.4,37.4]
vitals_tempc_36h_48h_before, median [Q1,Q3]                             7142     36.8 [36.6,37.1]     36.9 [36.4,37.4]
vitals_spo2, median [Q1,Q3]                                             1270     97.0 [95.0,99.0]     97.0 [95.0,99.0]
cbc_hemoglobin, median [Q1,Q3]                                          2615       9.9 [8.3,12.0]      10.0 [8.5,12.0]
cbc_hematocrit, median [Q1,Q3]                                          3021     30.2 [25.2,36.0]     31.0 [26.4,37.0]
cbc_mch, median [Q1,Q3]                                                 5079     29.9 [28.4,31.2]     29.9 [28.1,31.0]
cbc_mchc, median [Q1,Q3]                                                3403     33.0 [32.0,34.0]     32.7 [31.6,33.7]
cbc_mcv,